In [ ]:
import json, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors

import pytz 

from datetime import timedelta, datetime

from aquabyte.data_access_utils import RDSAccessUtils

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

In [ ]:
pen_id = 66
site_name = 'Seglberget'
pen_name = 'Pen 1'

In [ ]:
query = """
    select date, female_avg, female_moving_avg, moving_avg, moving_moving_avg, num_lati_fish, num_moving_avg_lati_fish
    from day_summaries a
    where a.pen_id = %i
    and a.date >= '2020-03-01'
    and a.date < '2020-04-12';
""" % (pen_id, )

day_summaries = rds_access_utils.extract_from_database(query)

day_summaries.index = day_summaries['date']
day_summaries = day_summaries.sort_index()

day_summaries

In [ ]:
query = """
    select captured_at, annotation_metadata 
    from annotations a
    where a.pen_id = %i
    and a.is_qa = true
    and a.is_skipped = false
    and a.captured_at > '2020-03-01'
    and a.captured_at < '2020-04-12';
""" % (pen_id, )

lice_counts = rds_access_utils.extract_from_database(query)

lice_counts.index = lice_counts['captured_at']
lice_counts = lice_counts.sort_index()

lice_counts

for index, lice_count in lice_counts.iterrows():
    liceCounts = lice_count['annotation_metadata']['liceCounts']

    lice_counts.ix[index, 'movingCountAdjusted'] = liceCounts['movingCountAdjusted']
    lice_counts.ix[index, 'adultFemaleCountAdjusted'] = liceCounts['adultFemaleCountAdjusted']
    lice_counts.ix[index, 'count'] = 1

lice_counts

In [ ]:
numFishs = [ 15, 30, 50 ]

moving_averages = []

tz = pytz.timezone('Europe/Oslo')

for numFish in numFishs:
    subsetPandas = pd.DataFrame()

    for index, date in enumerate(day_summaries.index):
        if index == 0:
            continue

        startDate = datetime.combine(day_summaries.index[index - 1], datetime.min.time()).replace(tzinfo=tz)
        endDate = datetime.combine(date, datetime.min.time()).replace(tzinfo=tz)

        lice_counts_subset = lice_counts[(startDate < lice_counts.index) & (endDate > lice_counts.index)]

        if len(lice_counts_subset) > 0:
            frac = min(1, numFish / len(lice_counts_subset))

            subset = lice_counts_subset.sample(frac = frac)

            subsetPandas = pd.concat([subsetPandas, subset])

    subsetPandas = subsetPandas.sort_index()

    columns = [ 'movingCountAdjusted', 'adultFemaleCountAdjusted' ]
    subsetPandas.index = pd.to_datetime(subsetPandas.captured_at, format='%Y-%m-%d')

    

    i = 5

    daily_avg_count = subsetPandas['count'].rolling('%iD' % (i, )).sum().resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
    daily_avg = subsetPandas[columns].rolling('%iD' % (i, )).mean().resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
    daily_center_avg_count = subsetPandas['count'].rolling('%iD' % (i, )).sum().shift(-24 * i / 2, freq='h').resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
    daily_center_avg = subsetPandas[columns].rolling('%iD' % (i, )).mean().shift(-24 * i / 2, freq='h').resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
    daily_avg['count'] = daily_avg_count
    daily_center_avg['count'] = daily_center_avg_count

    moving_averages.append((numFish, daily_avg, daily_center_avg))

In [ ]:
cmap = plt.cm.rainbow
norm = matplotlib.colors.Normalize(vmin=0, vmax=2)

fig, ax = plt.subplots(2)

fig.set_size_inches(15, 20)

minCount = 50

for index, moving_average in enumerate(moving_averages):
    numFish, daily_avg, daily_center_avg = moving_average
    
    subset = (daily_avg['count'] >= minCount)
    subset_center = (daily_center_avg.index >= np.min(daily_avg.index)) & (daily_center_avg['count'] >= minCount)
    ax[0].plot(daily_center_avg.index[subset_center], daily_center_avg['adultFemaleCountAdjusted'][subset_center], color = cmap(norm(index)), linestyle = '--', marker = '*', label = '%i fish 5 day Centered MA' % (numFish, ))
ax[0].plot(day_summaries.index, day_summaries['female_moving_avg'], linestyle = 'solid', linewidth = 5, marker = 'o', color = 'black', label = 'Current AF')
ax[0].set_title('%s %s Adult Female Counts' % (site_name, pen_name))
ax[0].set_xlabel('Date')
ax[0].set_ylabel('Adult Female Count')

#ax02 = ax[0].twinx() 
#ax02.plot(moving_average[1].index, moving_average[1]['count'], color = 'red', linestyle = '--', marker = 'o', label='Count')

for index, moving_average in enumerate(moving_averages):
    numFish, daily_avg, daily_center_avg = moving_average
    
    subset = (moving_average[1]['count'] >= minCount)
    subset_center = (daily_center_avg.index >= np.min(daily_avg.index)) & (daily_center_avg['count'] >= minCount)
    ax[1].plot(daily_center_avg.index[subset_center], daily_center_avg['movingCountAdjusted'][subset_center], color = cmap(norm(index)), linestyle = '--', marker = '*', label = '%i fish 5 day Centered MA' % (numFish, ))
ax[1].plot(day_summaries.index, day_summaries['moving_moving_avg'], linestyle = 'solid', linewidth = 5, marker = 'o', color = 'black', label = 'Current AF')
ax[1].set_title('%s %s Mobile Counts' % (site_name, pen_name))
ax[1].set_xlabel('Date')
ax[1].set_ylabel('Mobile Count')

ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
ax[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

plt.show()